In [ ]:
from astropy.io import fits
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pyfits

%matplotlib inline

starPosCCD = [1048+256,1024]

tab = [{'FName': 'IPHAS_GTC_DATA/DATA2_2017A/GTC12-17AMEX/OB0001/gtc_object_av_wl_flt_cal.fits', 'OName': 'IPHASXJ194645', 'SkyLeft':[835,960], 'SkyRight':[1570,1760], 'ObjectAreas':[[1262,1330],[1157,1240]]},#very wide, find sky from other imeage nearby
{'FName': 'IPHAS_GTC_DATA/DATA2_2017A/GTC12-17AMEX/OB0002/gtc_object_av_wl_flt_cal.fits', 'OName': 'Kn19', 'SkyLeft':[1094,1166], 'SkyRight':[1474,1588], 'ObjectAreas':[[1200,1228],[1249,1306],[1321,1340],[1358,1450]]},#looks good
{'FName': 'IPHAS_GTC_DATA/DATA2_2017A/GTC12-17AMEX/OB0003/gtc_object_av_wl_flt_cal.fits', 'OName': 'Kn45', 'SkyLeft':[1104,1228], 'SkyRight':[1320,1435], 'ObjectAreas':[[1238,1296]]},#okay, small
{'FName': 'IPHAS_GTC_DATA/DATA2_2017A/GTC12-17AMEX/OB0004/gtc_object_av_wl_flt_cal.fits', 'OName': 'DSHJ205811', 'SkyLeft':[1094,1257], 'SkyRight':[1325,1469], 'ObjectAreas':[[1263,1286]]},#okay, small
{'FName': 'IPHAS_GTC_DATA/DATA2_2017A/GTC12-17AMEX/OB0005/gtc_object_av_wl_flt_cal.fits', 'OName': 'DSHJ221013', 'SkyLeft':[680,780], 'SkyRight':[1626,1704], 'ObjectAreas':[[1090,1292],[1334,1380],[1404,1506],[1538,1554]]},#okay
{'FName': 'IPHAS_GTC_DATA/DATA2_2017A/GTC12-17AMEX/OB0006/gtc_object_av_wl_flt_cal.fits', 'OName': 'IPHASXJ194745', 'SkyLeft':[1092,1276], 'SkyRight':[1544,1756], 'ObjectAreas':[[1300,1396]]},#hmmm, run that and try with sky from other image
{'FName': 'IPHAS_GTC_DATA/DATA2_2017A/GTC12-17AMEX/OB0007/gtc_object_av_wl_flt_cal.fits', 'OName': 'Ou3', 'SkyLeft':[836,1028], 'SkyRight':[1536,1608], 'ObjectAreas':[[1133,1227],[1238,1306],[1316,1387],[1402,1496]]},#looks good, line at 5004
{'FName': 'IPHAS_GTC_DATA/DATA2_2017A/GTC12-17AMEX/OB0008/gtc_object_av_wl_flt_cal.fits', 'OName': 'Kn24', 'SkyLeft':[700,850], 'SkyRight':[1700,1812], 'ObjectAreas':[[916,1038],[1090,1180],[1198,1306],[1320,1556],[1566,1700]]},#looks good, line at 5004
{'FName': 'IPHAS_GTC_DATA/DATA2_2017A/GTC12-17AMEX/OB0009/gtc_object_av_wl_flt_cal.fits', 'OName': 'LDu1', 'SkyLeft':[440,570], 'SkyRight':[1837,1918], 'ObjectAreas':[[1090,1350],[1374,1569]]},#looks good, line at 5004 and 6582
{'FName': 'IPHAS_GTC_DATA/DATA2_2017A/GTC12-17AMEX/OB0010/gtc_object_av_wl_flt_cal.fits', 'OName': 'IPHASXJ193617', 'SkyLeft':[1231,1288], 'SkyRight':[1423,1479], 'ObjectAreas':[[1293,1308],[1316,1331]]},#looks good, pretty compact
{'FName': 'IPHAS_GTC_DATA/DATA2_2017A/GTC12-17AMEX/OB0011/gtc_object_av_wl_flt_cal.fits', 'OName': 'IPHASXJ194728', 'SkyLeft':[1090,1185], 'SkyRight':[1444,1555], 'ObjectAreas':[[1248,1368]]},#looks good, lines at 6716,...
{'FName': 'IPHAS_GTC_DATA/DATA2_2017A/GTC12-17AMEX/OB0012/gtc_object_av_wl_flt_cal.fits', 'OName': 'IPHASXJ194301', 'SkyLeft':[1139,1248], 'SkyRight':[1382,1513], 'ObjectAreas':[[1272,1325]]},#looks good, pretty compact, lines at 6585, 6562
{'FName': 'IPHAS_GTC_DATA/DATA2_2017A/GTC12-17AMEX/OB0012a/gtc_object_av_wl_flt_cal.fits', 'OName': 'IPHASXJ194301', 'SkyLeft':[1090,1222], 'SkyRight':[1357,1475], 'ObjectAreas':[[1284,1348]]},#looks good, pretty compact, lines at 6585, 6562
{'FName': 'IPHAS_GTC_DATA/DATA3_2017B/GTC8-17BMEX/OB0001/gtc_object_av_wl_flt_cal.fits', 'OName': 'PNG124.1-01.9', 'SkyLeft':[1090,1177], 'SkyRight':[1376,1449], 'ObjectAreas':[[1299,1350]]},#looks good, lines at 4859 and 6716
{'FName': 'IPHAS_GTC_DATA/DATA3_2017B/GTC8-17BMEX/OB0002/gtc_object_av_wl_flt_cal.fits', 'OName': 'IPHASXJ014238+600947', 'SkyLeft':[682,816], 'SkyRight':[1930,2013], 'ObjectAreas':[[864,953],[968,1038],[1091,1126],[1138,1303],[1318,1424],[1440,1720],[1759,1902]]},#looks good, lines at 6584
{'FName': 'IPHAS_GTC_DATA/DATA3_2017B/GTC8-17BMEX/OB0003/gtc_object_av_wl_flt_cal.fits', 'OName': 'IPHASXJ031058.8+624755', 'SkyLeft':[170,324], 'SkyRight':[1961,2024], 'ObjectAreas':[[363,457],[478,542],[559,576],[594,726],[743,786],[820,855],[872,897],[910,1038],[1090,1273],[1328,1760],[1777,1850],[1867,1909],[1918,1950]]},#looks good at 4957 and 6579 - with central compact area (?)
{'FName': 'IPHAS_GTC_DATA/DATA3_2017B/GTC8-17BMEX/OB0003a/gtc_object_av_wl_flt_cal.fits', 'OName': 'IPHASXJ031058.8+624755', 'SkyLeft':[170,324], 'SkyRight':[1961,2024], 'ObjectAreas':[[363,457],[478,542],[559,576],[594,726],[743,786],[820,855],[872,897],[910,1038],[1090,1273],[1328,1431],[1504,1756],[1777,1850],[1867,1909],[1918,1950]]},#looks good at 4957 and 6579 - without central compact area (?)
{'FName': 'IPHAS_GTC_DATA/DATA3_2017B/GTC8-17BMEX/OB0004/gtc_object_av_wl_flt_cal.fits', 'OName': 'IPHASXj040721.5+512422', 'SkyLeft':[231,522], 'SkyRight':[1842,2010], 'ObjectAreas':[[1197,1303],[1320,1450]]},#looks good, lines at 4959, 5004, 6580, 6716
{'FName': 'IPHAS_GTC_DATA/DATA3_2017B/GTC8-17BMEX/OB0005/gtc_object_av_wl_flt_cal.fits', 'OName': 'IPHASXJ051152.2+302751', 'SkyLeft':[167,380], 'SkyRight':[1872,2025], 'ObjectAreas':[[701,1035],[1124,1154],[1171,1300],[1325,1555],[1607,1880]]},#Halpha region?
{'FName': 'IPHAS_GTC_DATA/DATA3_2017B/GTC8-17BMEX/OB0006/gtc_object_av_wl_flt_cal.fits', 'OName': 'IPHASXJ053650.8+245616', 'SkyLeft':[300,556], 'SkyRight':[1778,1923], 'ObjectAreas':[[1201,1304],[1320,1474]]},#faint lines around 6716
{'FName': 'IPHAS_GTC_DATA/DATA3_2017B/GTC8-17BMEX/OB0009/gtc_object_av_wl_flt_cal.fits', 'OName': 'IPHASXJ055242.8+262116', 'SkyLeft':[1090,1248], 'SkyRight':[1675,1957], 'ObjectAreas':[[1265,1304],[1317,1466]]},#looks good, compact, 4959, 5008, 6563
{'FName': 'IPHAS_GTC_DATA/DATA3_2017B/GTC8-17BMEX/OB0013/gtc_object_av_wl_flt_cal.fits', 'OName': 'IPHASXJ185322.1+083018', 'SkyLeft':[622,808], 'SkyRight':[1771,1971], 'ObjectAreas':[[1491,1537]]},#shell visible at 5005
{'FName': 'IPHAS_GTC_DATA/DATA3_2017B/GTC8-17BMEX/OB0020/gtc_object_av_wl_flt_cal.fits', 'OName': 'IPHASXJ191306.1+025248', 'SkyLeft':[423,468], 'SkyRight':[1836,1878], 'ObjectAreas':[[846,870],[882,977],[985,1002],[1011,1038],[1090,1119],[1132,1183],[1196,1223],[1245,1304],[1346,1390],[1519,1534],[1565,1626],[1635,1696],[1714,1746],[1758,1806]]},#clear shell at 6583
{'FName': 'IPHAS_GTC_DATA/DATA3_2017B/GTC8-17BMEX/OB0026/gtc_object_av_wl_flt_cal.fits', 'OName': 'SNR060707', 'SkyLeft':[325,513], 'SkyRight':[1936,2021], 'ObjectAreas':[[888,958],[1101,1152],[1180,1210],[1245,1299],[1309,1350],[1368,1399],[1425,1478],[1491,1560]]},#lines at 6561, 6584, without extended regions near the center
{'FName': 'IPHAS_GTC_DATA/DATA3_2017B/GTC8-17BMEX/OB0026a/gtc_object_av_wl_flt_cal.fits', 'OName': 'SNR060707', 'SkyLeft':[325,513], 'SkyRight':[1936,2021], 'ObjectAreas':[[888,958],[1101,1299],[1309,1350],[1368,1399],[1425,1478],[1491,1560]]},#lines at 6561, 6584, without extended regions near the center
{'FName': 'IPHAS_GTC_DATA/DATA3_2017B/GTC8-17BMEX/OB0027/gtc_object_av_wl_flt_cal.fits', 'OName': 'SNR060715', 'SkyLeft':[517,800], 'SkyRight':[1718,2004], 'ObjectAreas':[[1256,1302],[1320,1388]]},#ok, some lines at 6563, 6717
{'FName': 'IPHAS_GTC_DATA/DATA3_2017B/GTC8-17BMEX/OB0028/gtc_object_av_wl_flt_cal.fits', 'OName': 'SNR060930', 'SkyLeft':[744,935], 'SkyRight':[1788,1927], 'ObjectAreas':[[1126,1242],[1258,1348],[1408,1555]]},#possible line at 6566
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0001/gtc_object_av_wl_flt_cal.fits', 'OName': 'KK26', 'SkyLeft':[680,920], 'SkyRight':[1730,2014], 'ObjectAreas':[[1112,1283],[1305,1559]]},#very nice blobs
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0002/gtc_object_av_wl_flt_cal.fits', 'OName': 'Kn34', 'SkyLeft':[800,940], 'SkyRight':[1468,1565], 'ObjectAreas':[[1163,1300],[1320,1360],[1394,1443]]},#ok, lines at 4861, 4960, 5006
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0003/gtc_object_av_wl_flt_cal.fits', 'OName': 'CR1', 'SkyLeft':[368,684], 'SkyRight':[1926,2025], 'ObjectAreas':[[1090,1341],[1358,1458],[1470,1530],[1537,1625]]},#ok, lines at 4959, 5005
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0004/gtc_object_av_wl_flt_cal.fits', 'OName': 'Kn11', 'SkyLeft':[1122,1194], 'SkyRight':[1427,1532], 'ObjectAreas':[[1278,1303],[1317,1340]]},#okay, compact, lines 6547, 6562, 6582
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0005/gtc_object_av_wl_flt_cal.fits', 'OName': 'HaWe3', 'SkyLeft':[643,1017], 'SkyRight':[1489,1628], 'ObjectAreas':[[1229,1301],[1320,1391]]},#okay, lines at 4958, 5005
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0006/gtc_object_av_wl_flt_cal.fits', 'OName': 'Kn48', 'SkyLeft':[770,874], 'SkyRight':[1442,1570], 'ObjectAreas':[[1216,1301],[1316,1332],[1345,1389],[1401,1420]]},#okay lines at 4861, 4958, 5006
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0007/gtc_object_av_wl_flt_cal.fits', 'OName': 'Kn58', 'SkyLeft':[1090,1190], 'SkyRight':[1526,1700], 'ObjectAreas':[[1195,1304],[1324,1430]]},#okay, lines at 4955, 5005
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0008/gtc_object_av_wl_flt_cal.fits', 'OName': 'Ou2', 'SkyLeft':[852,925], 'SkyRight':[1528,1604], 'ObjectAreas':[[1133,1367],[1397,1507]]},#okay, lines at 4955, 5005
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0009/gtc_object_av_wl_flt_cal.fits', 'OName': 'Abell68', 'SkyLeft':[1090,1168], 'SkyRight':[1432,1477], 'ObjectAreas':[[1188,1305],[1317,1368],[1379,1397]]},#okay, lines at 4684, 4861, 4958, 5005
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0010/gtc_object_av_wl_flt_cal.fits', 'OName': 'Kn24', 'SkyLeft':[747,939], 'SkyRight':[1893,2020], 'ObjectAreas':[[945,998],[1013,1036],[1090,1241],[1254,1303],[1338,1400],[1453,1558],[1581,1763]]},#hmmm, possible line at 6585
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0011/gtc_object_av_wl_flt_cal.fits', 'OName': 'IPHASXJ190333_Green_Slit', 'SkyLeft':[612,702], 'SkyRight':[1823,1855], 'ObjectAreas':[[1090,1125],[1164,1212],[1238,1252],[1267,1295],[1307,1353],[1367,1444],[1495,1516],[1530,1556],[1600,1650],[1665,1700]]},#faint line at 5005, 6565, 6585, 6829, 6862
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0012/gtc_object_av_wl_flt_cal.fits', 'OName': 'K1-6', 'SkyLeft':[314,732], 'SkyRight':[1680,1957], 'ObjectAreas':[[957,1036],[1090,1132],[1148,1243],[1388,1582]]},#okay, lines at 4861, 4959, 5005
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0013/gtc_object_av_wl_flt_cal.fits', 'OName': 'LDu1', 'SkyLeft':[407,717], 'SkyRight':[1847,2020], 'ObjectAreas':[[643,752],[762,807],[819,880],[1090,1166],[1182,1226],[1238,1304],[1315,1350],[1395,1452],[1465,1480],[1492,1550],[1561,1573],[1588,1673]]},
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0015/gtc_object_av_wl_flt_cal.fits', 'OName': 'IPHASXJ195627', 'SkyLeft':[1090,1178], 'SkyRight':[1438,1505], 'ObjectAreas':[[1259,1337]]},#Hmmm, faint line at 6563
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0016/gtc_object_av_wl_flt_cal.fits', 'OName': 'IRAS20084', 'SkyLeft':[1128,1206], 'SkyRight':[1681,1780], 'ObjectAreas':[[1290,1299],[1322,1329]]},#nothing really but HASH shows compact cloud in 432
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0017/gtc_object_av_wl_flt_cal.fits', 'OName': 'IPHASXJ201058', 'SkyLeft':[637,914], 'SkyRight':[1671,1781], 'ObjectAreas':[[1213,1268],[1286,1340]]},#faint lines at 6565 and 6585
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0018/gtc_object_av_wl_flt_cal.fits', 'OName': 'Kn36', 'SkyLeft':[366,547], 'SkyRight':[1884,1940], 'ObjectAreas':[[1188,1300],[1345,1361],[1398,1420]]},#ok lines at 4956, 5005
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0019/gtc_object_av_wl_flt_cal.fits', 'OName': '', 'SkyLeft':[612,0], 'SkyRight':[0,0], 'ObjectAreas':[[0,0],[0,0]]},#
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0023/gtc_object_av_wl_flt_cal.fits', 'OName': 'Kn25', 'SkyLeft':[445,489], 'SkyRight':[1614,1658], 'ObjectAreas':[[1171,1212],[1265,1283],[1331,1345],[1398,1433]]},#lines at 5005
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0020/gtc_object_av_wl_flt_cal.fits', 'OName': '', 'SkyLeft':[612,0], 'SkyRight':[0,0], 'ObjectAreas':[[0,0],[0,0]]},#
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0024/gtc_object_av_wl_flt_cal.fits', 'OName': 'We1-10', 'SkyLeft':[369,450], 'SkyRight':[1922,2023], 'ObjectAreas':[[752,839],[860,978],[994,1020],[1090,1195],[1206,1295],[1325,1402],[1417,1472],[1504,1554],[1568,1618],[1633,1658],[1675,1845]]},# line at 4959
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0025/gtc_object_av_wl_flt_cal.fits', 'OName': 'KTC1', 'SkyLeft':[1090,1190], 'SkyRight':[1390,1505], 'ObjectAreas':[[1244,1301],[1321,1375]]},#easy, lines at 4957 and 5005
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0026/gtc_object_av_wl_flt_cal.fits', 'OName': 'Kn30', 'SkyLeft':[1090,1260], 'SkyRight':[1388,1506], 'ObjectAreas':[[1301,1329],[1338,1364]]},#easy, lines at 4859, 4957, 5005
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0027/gtc_object_av_wl_flt_cal.fits', 'OName': 'Kn31', 'SkyLeft':[496,900], 'SkyRight':[1611,1720], 'ObjectAreas':[[1167,1222],[1238,1301],[1316,1495]]},#okay, lines at 4959, 5005
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0028/gtc_object_av_wl_flt_cal.fits', 'OName': '', 'SkyLeft':[612,0], 'SkyRight':[0,0], 'ObjectAreas':[[0,0],[0,0]]},#
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0029/gtc_object_av_wl_flt_cal.fits', 'OName': 'IPHASXJ230323', 'SkyLeft':[802,1031], 'SkyRight':[1696,1866], 'ObjectAreas':[[1150,1222],[1248,1397]]},#faint line at 6561
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0030/gtc_object_av_wl_flt_cal.fits', 'OName': 'IPHASXJ010133', 'SkyLeft':[520,716], 'SkyRight':[1685,1794], 'ObjectAreas':[[1100,1242],[1259,1375]]},#line at 4859
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0031/gtc_object_av_wl_flt_cal.fits', 'OName': 'Ju1', 'SkyLeft':[185,390], 'SkyRight':[1819,1983], 'ObjectAreas':[[790,880],[901,974],[1004,1035],[1090,1201],[1222,1238],[1261,1301],[1344,1414],[1432,1495],[1515,1543],[1600,1632],[1728,1814]]},#line at 1960
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0032/gtc_object_av_wl_flt_cal.fits', 'OName': 'Kn51', 'SkyLeft':[391,688], 'SkyRight':[1847,1978], 'ObjectAreas':[[1200,1302],[1317,1448]]},#line at 5005
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0033/gtc_object_av_wl_flt_cal.fits', 'OName': 'FsMv1', 'SkyLeft':[695,817], 'SkyRight':[1650,1809], 'ObjectAreas':[[1131,1164]]},#difficult, possible line at 6581, without extended objects
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0033a/gtc_object_av_wl_flt_cal.fits', 'OName': 'FsMv1', 'SkyLeft':[695,817], 'SkyRight':[1650,1809], 'ObjectAreas':[[801,891],[1131,1164]]},#difficult, possible line at 6581, with left extended object
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0033b/gtc_object_av_wl_flt_cal.fits', 'OName': 'FsMv1', 'SkyLeft':[695,817], 'SkyRight':[1650,1809], 'ObjectAreas':[[1131,1164],[1205,1306]]},#difficult, possible line at 6581, with left extended object
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0034/gtc_object_av_wl_flt_cal.fits', 'OName': 'KnJ0240', 'SkyLeft':[1142,1244], 'SkyRight':[1395,1530], 'ObjectAreas':[[1284,1303],[1327,1351]]},#compact, lines at 4988, 5090, 5137
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0035/gtc_object_av_wl_flt_cal.fits', 'OName': 'KnJ1857', 'SkyLeft':[810,913], 'SkyRight':[1874,2014], 'ObjectAreas':[[1221,1293],[1335,1374]]},#can't see lines just a possibly extended (compact) object in the center
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0036/gtc_object_av_wl_flt_cal.fits', 'OName': 'IPHASXJ214032', 'SkyLeft':[1090,1250], 'SkyRight':[1552,1661], 'ObjectAreas':[[1312,1356]]},#lines at 4961 and 5005, no central star visible
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0037/gtc_object_av_wl_flt_cal.fits', 'OName': 'DSHJ192315', 'SkyLeft':[1185,1214], 'SkyRight':[1687,1748], 'ObjectAreas':[[1277,1299],[1318,1329]]},#lines at 4961 and 5005, pretty compact object
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0038/gtc_object_av_wl_flt_cal.fits', 'OName': 'DSHJ204858', 'SkyLeft':[908,965], 'SkyRight':[1745,1890], 'ObjectAreas':[[1090,1284],[1430,1556]]},#lines at 4860, 4958, 5005, central object gets wider in some wavelength ranges
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0039/gtc_object_av_wl_flt_cal.fits', 'OName': 'Kn20', 'SkyLeft':[878,964], 'SkyRight':[1570,1606], 'ObjectAreas':[[1296,1332]]},#lines at 6563 and 6582, only on right side of a star (no central star visible?)
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0040/gtc_object_av_wl_flt_cal.fits', 'OName': 'DSHJ195813', 'SkyLeft':[708,799], 'SkyRight':[1603,1655], 'ObjectAreas':[[1214,1257],[1288,1300],[1320,1354],[1383,1394]]},#lines at 4959, 5005
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0042/gtc_object_av_wl_flt_cal.fits', 'OName': 'Kn59', 'SkyLeft':[1090,1185], 'SkyRight':[1412,1511], 'ObjectAreas':[[1285,1305],[1320,1340]]},#compact, lines at 4861, 4959, 5005
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0043/gtc_object_av_wl_flt_cal.fits', 'OName': '', 'SkyLeft':[612,0], 'SkyRight':[0,0], 'ObjectAreas':[[0,0],[0,0]]},#
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0044/gtc_object_av_wl_flt_cal.fits', 'OName': 'Alves1', 'SkyLeft':[286,446], 'SkyRight':[1752,1939], 'ObjectAreas':[[1170,1297],[1324,1471]]},#nothing, without central object
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0044a/gtc_object_av_wl_flt_cal.fits', 'OName': 'Alves1', 'SkyLeft':[286,446], 'SkyRight':[1752,1939], 'ObjectAreas':[[1170,1471]]},#nothing, with central object
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0045/gtc_object_av_wl_flt_cal.fits', 'OName': 'DSHJ205943', 'SkyLeft':[1090,1143], 'SkyRight':[1348,1403], 'ObjectAreas':[[1291,1306],[1317,1329]]},#compact, lines at 4858, 4956, 5005
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0046/gtc_object_av_wl_flt_cal.fits', 'OName': 'Ou5', 'SkyLeft':[1123,1257], 'SkyRight':[1440,1512], 'ObjectAreas':[[1274,1323],[1342,1385]]},#lines at 4861, 4959, 5005, 5873
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0048/gtc_object_av_wl_flt_cal.fits', 'OName': 'DSHJ062355', 'SkyLeft':[811,1028], 'SkyRight':[1677,1790], 'ObjectAreas':[[1090,1139],[1154,1213],[1225,1278],[1301,1334],[1361,1434],[1462,1608]]},#lines at 4959, 5005
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0049/gtc_object_av_wl_flt_cal.fits', 'OName': '', 'SkyLeft':[612,0], 'SkyRight':[0,0], 'ObjectAreas':[[0,0],[0,0]]},#
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0052/gtc_object_av_wl_flt_cal.fits', 'OName': 'Ou2', 'SkyLeft':[831,909], 'SkyRight':[1800,1984], 'ObjectAreas':[[1128,1263],[1277,1300],[1318,1327],[1338,1344],[1381,1483]]},#lines at 4684, 4957, 5005
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0055/gtc_object_av_wl_flt_cal.fits', 'OName': 'We2-260', 'SkyLeft':[512,772], 'SkyRight':[1677,1777], 'ObjectAreas':[[871,1038],[1090,1400],[1426,1560],[1567,1657]]},#line at 6582
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0056/gtc_object_av_wl_flt_cal.fits', 'OName': 'Si1-2', 'SkyLeft':[714,954], 'SkyRight':[1524,1578], 'ObjectAreas':[[1167,1304],[1335,1369],[1386,1406],[1419,1436]]},#lines at 4959, 5005
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0057/gtc_object_av_wl_flt_cal.fits', 'OName': 'IPHASXJ023538', 'SkyLeft':[719,956], 'SkyRight':[1784,1987], 'ObjectAreas':[[1247,1321],[1370,1449]]},#faint continuum both sides of the central faint object (not included)
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0062/gtc_object_av_wl_flt_cal.fits', 'OName': 'DSHJ202907', 'SkyLeft':[926,1034], 'SkyRight':[1600,1847], 'ObjectAreas':[[1227,1274]]},#lines at 4860, 4958, 5005, bright star offset to center (not included)
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0065/gtc_object_av_wl_flt_cal.fits', 'OName': 'Pa30', 'SkyLeft':[839,935], 'SkyRight':[1740,1823], 'ObjectAreas':[[1323,1344],[1389,1400]]},#possible doublet at 6697, 6711 and redshifted at 6737, 6750
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0067/gtc_object_av_wl_flt_cal.fits', 'OName': '', 'SkyLeft':[612,0], 'SkyRight':[0,0], 'ObjectAreas':[[0,0],[0,0]]},#
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0068/gtc_object_av_wl_flt_cal.fits', 'OName': 'HaWe15', 'SkyLeft':[394,486], 'SkyRight':[1964,2025], 'ObjectAreas':[[711,766],[796,916],[936,997],[1020,1038],[1131,1208],[1365,1565],[1707,1765],[1799,1822]]},#extended object with lines at 4957, 5005,...
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0069/gtc_object_av_wl_flt_cal.fits', 'OName': 'We2-5', 'SkyLeft':[710,772], 'SkyRight':[1925,1973], 'ObjectAreas':[[879,1037],[1090,1125],[1143,1233],[1251,1297],[1397,1446],[1514,1711],[1742,1786],[1846,1907]]},#lines at 6546, 6562, 6582
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0070/gtc_object_av_wl_flt_cal.fits', 'OName': 'KLSS2-8', 'SkyLeft':[702,1017], 'SkyRight':[1560,1740], 'ObjectAreas':[[1136,1299],[1325,1409],[1437,1479]]},#line at 5005
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0073/gtc_object_av_wl_flt_cal.fits', 'OName': 'PM1-305', 'SkyLeft':[1149,1211], 'SkyRight':[1625,1663], 'ObjectAreas':[[1283,1303],[1322,1359]]},#lines at 4860, 4959, 5005
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0079/gtc_object_av_wl_flt_cal.fits', 'OName': 'Kn33', 'SkyLeft':[778,891], 'SkyRight':[1791,1879], 'ObjectAreas':[[1305,1351],[1376,1418]]},#lines at 4861, 4959, 5005
{'FName': 'IPHAS_GTC_DATA/DATA4_2016A/GTC4-16AMEX/OB0080/gtc_object_av_wl_flt_cal.fits', 'OName': 'KKR62', 'SkyLeft':[860,962], 'SkyRight':[1577,1769], 'ObjectAreas':[[1195,1295],[1318,1378]]},#line at 5005
{'FName': 'IPHAS_GTC_DATA/DATA4_2018A/GTC11-18AMEX/OB0001/gtc_object_av_wl_flt_cal.fits', 'OName': 'IPHASXJ191306.1+025248', 'SkyLeft':[429,475], 'SkyRight':[1780,1837], 'ObjectAreas':[[929,972],[992,1038],[1090,1116],[1157,1196],[1234,1288],[1322,1348],[1376,1403],[1471,1526],[1640,1665],[1688,1714]]},#lines at 4861, 5005, 5200
{'FName': 'IPHAS_GTC_DATA/DATA4_2018A/GTC11-18AMEX/OB0003/gtc_object_av_wl_flt_cal.fits', 'OName': 'IPHASXJ190543.8+064413', 'SkyLeft':[717,921], 'SkyRight':[1567,1756], 'ObjectAreas':[[1267,1338]]},#lines at 6550, 6563, 6583, no central object visible
{'FName': 'IPHAS_GTC_DATA/DATA4_2018A/GTC11-18AMEX/OB0004/gtc_object_av_wl_flt_cal.fits', 'OName': 'IPHASXJ191058.9+040350', 'SkyLeft':[840,920], 'SkyRight':[1608,1674], 'ObjectAreas':[[1277,1368]]},#lines at 4957, 5005, no central object visible
{'FName': 'IPHAS_GTC_DATA/DATA4_2018A/GTC11-18AMEX/OB0005/gtc_object_av_wl_flt_cal.fits', 'OName': 'IPHASXJ191104.8+060845', 'SkyLeft':[786,1019], 'SkyRight':[1411,1512], 'ObjectAreas':[[1277,1325],[1340,1362]]},#lines at 4959, 5005
{'FName': 'IPHAS_GTC_DATA/DATA4_2018A/GTC11-18AMEX/OB0007/gtc_object_av_wl_flt_cal.fits', 'OName': 'IPHASXJ191707.3+020010', 'SkyLeft':[670,708], 'SkyRight':[1951,2005], 'ObjectAreas':[[1230,1271],[1300,1317],[1351,1385],[1420,1453]]},#lines at 4860 and 5005
{'FName': 'IPHAS_GTC_DATA/DATA4_2018A/GTC11-18AMEX/OB0008/gtc_object_av_wl_flt_cal.fits', 'OName': 'IPHASXJ191918.8+171148', 'SkyLeft':[1090,1186], 'SkyRight':[1451,1557], 'ObjectAreas':[[1251,1300],[1321,1331],[1340,1395]]}]#lines at 6548, 6561, 6583

i=1
for obs in tab:
    image_file = os.path.join('/Volumes/obiwan/azuri/spectra/IPHAS_GTC/',obs['FName'])
    hdulist = pyfits.open(image_file)
    header = hdulist[0].header
    obs['OName'] = header['OBJECT']
    print obs
    image_data = fits.getdata(image_file)
    print 'type(image_data) = ',type(image_data)
    print 'type(image_data[1000,1000]) = ',type(image_data[1000,1000])
    print 'len(image_data) = ',len(image_data)
    print 'image_data.shape = ',image_data.shape
    skyLeft = image_data[:,obs['SkyLeft'][0]:obs['SkyLeft'][1]]
    print 'skyLeft.shape = ',skyLeft.shape
    skyRight = image_data[:,obs['SkyRight'][0]:obs['SkyRight'][1]]
    print 'skyRight.shape = ',skyRight.shape
    nCols = 0
    for area in obs['ObjectAreas']:
        nCols += area[1]-area[0]
    print 'nCols = ',nCols
    obsCols = np.ndarray(shape=(skyLeft.shape[0],nCols), dtype=np.float32)
    nCols = 0
    for area in obs['ObjectAreas']:
        print 'image_data[1000,',area[0],':',area[1],'] = ',image_data[1000,area[0]:area[1]]
        obsCols[:,nCols:nCols+area[1]-area[0]] = image_data[:,area[0]:area[1]]
        nCols += area[1]-area[0]
    print 'obsCols[1000,:] = ',obsCols[0,:]
#    plt.imshow(image_data, cmap='gray')
#    plt.colorbar()
#    i+=1
#    if i == 3:
#        STOP
#print tab

In [15]:
# --- fix format of std table for G158-100 in /iraf/iraf/noao/lib/onedstds/esocal/
fname = '/iraf/iraf/noao/lib/onedstds/esocal/g158100.dat'
fcontent = []
with open(fname, "r") as f:
    for line in f:
        line = line.strip().split()
        print 'line = <',line,'>'
        fcontent.append([float(line[0]),float(line[1]),float(line[3])])
print 'len(fcontent) = ',len(fcontent)
print 'len(fcontent[1]) = ',len(fcontent[1])
print 'fcontent[1] = ',fcontent[1]
with open(fname,'w') as f:
    f.write("# G158-100\n")
    for cont in fcontent:
        print 'cont = ',cont
        f.write('%7.2f %7.3f %4.1f\n' % (cont[0], cont[1], cont[2]))

line = < ['3200.0000', '0.20050E+02', '0.68486E+00', '7.0'] >
line = < ['3201.0000', '0.21021E+02', '0.71845E+00', '7.0'] >
line = < ['3202.0000', '0.21696E+02', '0.74200E+00', '7.0'] >
line = < ['3203.0000', '0.21484E+02', '0.73519E+00', '7.0'] >
line = < ['3204.0000', '0.21176E+02', '0.72510E+00', '7.0'] >
line = < ['3205.0000', '0.19933E+02', '0.68296E+00', '7.0'] >
line = < ['3206.0000', '0.18797E+02', '0.64447E+00', '7.0'] >
line = < ['3207.0000', '0.19981E+02', '0.68549E+00', '7.0'] >
line = < ['3208.0000', '0.21279E+02', '0.73046E+00', '7.0'] >
line = < ['3209.0000', '0.21882E+02', '0.75163E+00', '7.0'] >
line = < ['3210.0000', '0.22439E+02', '0.77126E+00', '7.0'] >
line = < ['3211.0000', '0.22077E+02', '0.75928E+00', '7.0'] >
line = < ['3212.0000', '0.21502E+02', '0.73995E+00', '7.0'] >
line = < ['3213.0000', '0.21488E+02', '0.73995E+00', '7.0'] >
line = < ['3214.0000', '0.21793E+02', '0.75093E+00', '7.0'] >
line = < ['3215.0000', '0.22103E+02', '0.76208E+00', '7.0'] >
line = <

line = < ['3784.0000', '0.29931E+02', '0.14295E+01', '7.0'] >
line = < ['3785.0000', '0.29777E+02', '0.14230E+01', '7.0'] >
line = < ['3786.0000', '0.29353E+02', '0.14035E+01', '7.0'] >
line = < ['3787.0000', '0.29042E+02', '0.13893E+01', '7.0'] >
line = < ['3788.0000', '0.28920E+02', '0.13842E+01', '7.0'] >
line = < ['3789.0000', '0.28799E+02', '0.13791E+01', '7.0'] >
line = < ['3790.0000', '0.28863E+02', '0.13829E+01', '7.0'] >
line = < ['3791.0000', '0.28901E+02', '0.13855E+01', '7.0'] >
line = < ['3792.0000', '0.29207E+02', '0.14009E+01', '7.0'] >
line = < ['3793.0000', '0.29462E+02', '0.14138E+01', '7.0'] >
line = < ['3794.0000', '0.29582E+02', '0.14204E+01', '7.0'] >
line = < ['3795.0000', '0.29566E+02', '0.14204E+01', '7.0'] >
line = < ['3796.0000', '0.29415E+02', '0.14138E+01', '7.0'] >
line = < ['3797.0000', '0.29157E+02', '0.14022E+01', '7.0'] >
line = < ['3798.0000', '0.29141E+02', '0.14022E+01', '7.0'] >
line = < ['3799.0000', '0.29395E+02', '0.14151E+01', '7.0'] >
line = <

line = < ['4282.0000', '0.35639E+02', '0.21797E+01', '7.0'] >
line = < ['4283.0000', '0.35296E+02', '0.21597E+01', '7.0'] >
line = < ['4284.0000', '0.35053E+02', '0.21459E+01', '7.0'] >
line = < ['4285.0000', '0.34875E+02', '0.21360E+01', '7.0'] >
line = < ['4286.0000', '0.34731E+02', '0.21281E+01', '7.0'] >
line = < ['4287.0000', '0.34619E+02', '0.21223E+01', '7.0'] >
line = < ['4288.0000', '0.34412E+02', '0.21106E+01', '7.0'] >
line = < ['4289.0000', '0.34207E+02', '0.20989E+01', '7.0'] >
line = < ['4290.0000', '0.33908E+02', '0.20816E+01', '7.0'] >
line = < ['4291.0000', '0.33613E+02', '0.20644E+01', '7.0'] >
line = < ['4292.0000', '0.33289E+02', '0.20455E+01', '7.0'] >
line = < ['4293.0000', '0.32847E+02', '0.20193E+01', '7.0'] >
line = < ['4294.0000', '0.32411E+02', '0.19934E+01', '7.0'] >
line = < ['4295.0000', '0.32129E+02', '0.19770E+01', '7.0'] >
line = < ['4296.0000', '0.32025E+02', '0.19715E+01', '7.0'] >
line = < ['4297.0000', '0.31863E+02', '0.19625E+01', '7.0'] >
line = <

line = < ['4996.0000', '0.41646E+02', '0.34674E+01', '13.0'] >
line = < ['4998.0000', '0.41613E+02', '0.34674E+01', '13.0'] >
line = < ['5000.0000', '0.41618E+02', '0.34706E+01', '13.0'] >
line = < ['5002.0000', '0.41585E+02', '0.34706E+01', '13.0'] >
line = < ['5004.0000', '0.41628E+02', '0.34770E+01', '13.0'] >
line = < ['5006.0000', '0.41595E+02', '0.34770E+01', '13.0'] >
line = < ['5008.0000', '0.41562E+02', '0.34770E+01', '13.0'] >
line = < ['5010.0000', '0.41605E+02', '0.34834E+01', '13.0'] >
line = < ['5012.0000', '0.41572E+02', '0.34834E+01', '13.0'] >
line = < ['5014.0000', '0.41654E+02', '0.34930E+01', '13.0'] >
line = < ['5016.0000', '0.41697E+02', '0.34995E+01', '13.0'] >
line = < ['5018.0000', '0.41741E+02', '0.35059E+01', '13.0'] >
line = < ['5020.0000', '0.41707E+02', '0.35059E+01', '13.0'] >
line = < ['5022.0000', '0.41674E+02', '0.35059E+01', '13.0'] >
line = < ['5024.0000', '0.41641E+02', '0.35059E+01', '13.0'] >
line = < ['5026.0000', '0.41608E+02', '0.35059E+01', '1

line = < ['6146.0000', '0.38303E+02', '0.48261E+01', '13.0'] >
line = < ['6148.0000', '0.38278E+02', '0.48261E+01', '13.0'] >
line = < ['6150.0000', '0.38253E+02', '0.48261E+01', '13.0'] >
line = < ['6152.0000', '0.38299E+02', '0.48350E+01', '13.0'] >
line = < ['6154.0000', '0.38380E+02', '0.48484E+01', '13.0'] >
line = < ['6156.0000', '0.38426E+02', '0.48574E+01', '13.0'] >
line = < ['6158.0000', '0.38401E+02', '0.48574E+01', '13.0'] >
line = < ['6160.0000', '0.38447E+02', '0.48663E+01', '13.0'] >
line = < ['6162.0000', '0.38528E+02', '0.48798E+01', '13.0'] >
line = < ['6164.0000', '0.38574E+02', '0.48888E+01', '13.0'] >
line = < ['6166.0000', '0.38549E+02', '0.48888E+01', '13.0'] >
line = < ['6168.0000', '0.38595E+02', '0.48978E+01', '13.0'] >
line = < ['6170.0000', '0.38606E+02', '0.49023E+01', '13.0'] >
line = < ['6172.0000', '0.38652E+02', '0.49113E+01', '13.0'] >
line = < ['6174.0000', '0.38627E+02', '0.49113E+01', '13.0'] >
line = < ['6176.0000', '0.38602E+02', '0.49113E+01', '1

...

cont =  [3240.0, 21.21, 7.0]
cont =  [3241.0, 21.871, 7.0]
cont =  [3242.0, 22.552, 7.0]
cont =  [3243.0, 22.622, 7.0]
cont =  [3244.0, 22.608, 7.0]
cont =  [3245.0, 22.594, 7.0]
cont =  [3246.0, 22.642, 7.0]
cont =  [3247.0, 23.134, 7.0]
cont =  [3248.0, 23.988, 7.0]
cont =  [3249.0, 24.262, 7.0]
cont =  [3250.0, 23.717, 7.0]
cont =  [3251.0, 23.463, 7.0]
cont =  [3252.0, 23.907, 7.0]
cont =  [3253.0, 24.202, 7.0]
cont =  [3254.0, 24.12, 7.0]
cont =  [3255.0, 24.039, 7.0]
cont =  [3256.0, 23.848, 7.0]
cont =  [3257.0, 23.658, 7.0]
cont =  [3258.0, 22.873, 7.0]
cont =  [3259.0, 22.093, 7.0]
cont =  [3260.0, 22.74, 7.0]
cont =  [3261.0, 23.797, 7.0]
cont =  [3262.0, 23.892, 7.0]
cont =  [3263.0, 23.442, 7.0]
cont =  [3264.0, 23.106, 7.0]
cont =  [3265.0, 22.754, 7.0]
cont =  [3266.0, 22.824, 7.0]
cont =  [3267.0, 23.514, 7.0]
cont =  [3268.0, 23.958, 7.0]
cont =  [3269.0, 23.878, 7.0]
cont =  [3270.0, 23.863, 7.0]
cont =  [3271.0, 24.247, 7.0]
cont =  [3272.0, 24.615, 7.0]
cont =  [3273

cont =  [4045.0, 35.99, 7.0]
cont =  [4046.0, 36.205, 7.0]
cont =  [4047.0, 36.254, 7.0]
cont =  [4048.0, 36.236, 7.0]
cont =  [4049.0, 36.318, 7.0]
cont =  [4050.0, 36.3, 7.0]
cont =  [4051.0, 36.282, 7.0]
cont =  [4052.0, 36.398, 7.0]
cont =  [4053.0, 36.447, 7.0]
cont =  [4054.0, 36.665, 7.0]
cont =  [4055.0, 36.884, 7.0]
cont =  [4056.0, 36.9, 7.0]
cont =  [4057.0, 36.848, 7.0]
cont =  [4058.0, 36.525, 7.0]
cont =  [4059.0, 36.206, 7.0]
cont =  [4060.0, 36.022, 7.0]
cont =  [4061.0, 35.938, 7.0]
cont =  [4062.0, 36.053, 7.0]
cont =  [4063.0, 36.302, 7.0]
cont =  [4064.0, 36.519, 7.0]
cont =  [4065.0, 36.4, 7.0]
cont =  [4066.0, 36.382, 7.0]
cont =  [4067.0, 36.431, 7.0]
cont =  [4068.0, 36.447, 7.0]
cont =  [4069.0, 36.496, 7.0]
cont =  [4070.0, 36.545, 7.0]
cont =  [4071.0, 36.696, 7.0]
cont =  [4072.0, 36.847, 7.0]
cont =  [4073.0, 36.999, 7.0]
cont =  [4074.0, 37.186, 7.0]
cont =  [4075.0, 37.339, 7.0]
cont =  [4076.0, 37.321, 7.0]
cont =  [4077.0, 37.406, 7.0]
cont =  [4078.0, 

cont =  [5010.0, 41.605, 13.0]
cont =  [5012.0, 41.572, 13.0]
cont =  [5014.0, 41.654, 13.0]
cont =  [5016.0, 41.697, 13.0]
cont =  [5018.0, 41.741, 13.0]
cont =  [5020.0, 41.707, 13.0]
cont =  [5022.0, 41.674, 13.0]
cont =  [5024.0, 41.641, 13.0]
cont =  [5026.0, 41.608, 13.0]
cont =  [5028.0, 41.575, 13.0]
cont =  [5030.0, 41.542, 13.0]
cont =  [5032.0, 41.509, 13.0]
cont =  [5034.0, 41.476, 13.0]
cont =  [5036.0, 41.443, 13.0]
cont =  [5038.0, 41.486, 13.0]
cont =  [5040.0, 41.491, 13.0]
cont =  [5042.0, 41.459, 13.0]
cont =  [5044.0, 41.426, 13.0]
cont =  [5046.0, 41.469, 13.0]
cont =  [5048.0, 41.513, 13.0]
cont =  [5050.0, 41.48, 13.0]
cont =  [5052.0, 41.485, 13.0]
cont =  [5054.0, 41.529, 13.0]
cont =  [5056.0, 41.573, 13.0]
cont =  [5058.0, 41.54, 13.0]
cont =  [5060.0, 41.583, 13.0]
cont =  [5062.0, 41.55, 13.0]
cont =  [5064.0, 41.556, 13.0]
cont =  [5066.0, 41.6, 13.0]
cont =  [5068.0, 41.567, 13.0]
cont =  [5070.0, 41.534, 13.0]
cont =  [5072.0, 41.501, 13.0]
cont =  [5074

cont =  [6534.0, 35.161, 13.0]
cont =  [6536.0, 35.01, 13.0]
cont =  [6538.0, 34.892, 13.0]
cont =  [6540.0, 34.711, 13.0]
cont =  [6542.0, 34.467, 13.0]
cont =  [6544.0, 34.224, 13.0]
cont =  [6546.0, 33.921, 13.0]
cont =  [6548.0, 33.682, 13.0]
cont =  [6550.0, 33.415, 13.0]
cont =  [6552.0, 33.18, 13.0]
cont =  [6554.0, 33.038, 13.0]
cont =  [6556.0, 32.866, 13.0]
cont =  [6558.0, 32.785, 13.0]
cont =  [6560.0, 32.826, 13.0]
cont =  [6562.0, 32.806, 13.0]
cont =  [6564.0, 32.937, 13.0]
cont =  [6566.0, 33.069, 13.0]
cont =  [6568.0, 33.262, 13.0]
cont =  [6570.0, 33.457, 13.0]
cont =  [6572.0, 33.715, 13.0]
cont =  [6574.0, 33.975, 13.0]
cont =  [6576.0, 34.206, 13.0]
cont =  [6578.0, 34.406, 13.0]
cont =  [6580.0, 34.671, 13.0]
cont =  [6582.0, 34.81, 13.0]
cont =  [6584.0, 34.917, 13.0]
cont =  [6586.0, 34.993, 13.0]
cont =  [6588.0, 35.1, 13.0]
cont =  [6590.0, 35.112, 13.0]
cont =  [6592.0, 35.155, 13.0]
cont =  [6594.0, 35.198, 13.0]
cont =  [6596.0, 35.177, 13.0]
cont =  [6598

cont =  [7648.0, 30.912, 13.0]
cont =  [7650.0, 30.924, 13.0]
cont =  [7652.0, 30.908, 13.0]
cont =  [7654.0, 30.892, 13.0]
cont =  [7656.0, 30.876, 13.0]
cont =  [7658.0, 30.888, 13.0]
cont =  [7660.0, 30.872, 13.0]
cont =  [7662.0, 30.856, 13.0]
cont =  [7664.0, 30.84, 13.0]
cont =  [7666.0, 30.852, 13.0]
cont =  [7668.0, 30.836, 13.0]
cont =  [7670.0, 30.82, 13.0]
cont =  [7672.0, 30.804, 13.0]
cont =  [7674.0, 30.816, 13.0]
cont =  [7676.0, 30.8, 13.0]
cont =  [7678.0, 30.784, 13.0]
cont =  [7680.0, 30.768, 13.0]
cont =  [7682.0, 30.78, 13.0]
cont =  [7684.0, 30.764, 13.0]
cont =  [7686.0, 30.748, 13.0]
cont =  [7688.0, 30.761, 13.0]
cont =  [7690.0, 30.745, 13.0]
cont =  [7692.0, 30.7, 13.0]
cont =  [7694.0, 30.713, 13.0]
cont =  [7696.0, 30.668, 13.0]
cont =  [7698.0, 30.596, 13.0]
cont =  [7700.0, 30.58, 13.0]
cont =  [7702.0, 30.564, 13.0]
cont =  [7704.0, 30.548, 13.0]
cont =  [7706.0, 30.532, 13.0]
cont =  [7708.0, 30.601, 13.0]
cont =  [7710.0, 30.642, 13.0]
cont =  [7712.0,